In [ ]:
from erddapy import ERDDAP
import json
import yaml
import copy

base_erddap_url = "https://www.smartatlantic.ca/erddap"
output_results = "datasets.yml"
default_maxResponseTime = 5000
default_assignees = ["sjbruce","RichardJKelly"]

source_file = ".upptimerc.yml"

In [ ]:
e = ERDDAP(
    server=base_erddap_url,
    protocol="tabledap",
    response="json"
)

e.dataset_id = "allDatasets"

e.constraints = {
    "cdm_data_type=": "TimeSeries",
    "testOutOfDate!=": "",
}

e.variables = [
    "datasetID",
    "testOutOfDate",
    "cdm_data_type",
    "minTime",
    "maxTime",
    "title"
]

df = e.to_pandas()

In [ ]:
json_output = json.loads(df.to_json())

output = []

for row in json_output["datasetID"]:
    print(json_output["datasetID"][row])

    dataset_url = f"{base_erddap_url}/tabledap/{json_output['datasetID'][row]}.json?time&time>={json_output['testOutOfDate'][row]}&distinct()"

    dataset = {
        "name":json_output['title'][row],
        "url":dataset_url,
        "maxResponseTime": copy.deepcopy(default_maxResponseTime),
        "assignees": copy.deepcopy(default_assignees)
    }
    output.append(dataset)


In [ ]:
with open(source_file, mode="r") as yaml_file:
    source_yaml = yaml.safe_load(yaml_file)

source_yaml["sites"] = source_yaml["sites"] + output

with open(output_results, mode="wt", encoding="utf-8") as yaml_file:
    yaml.dump(source_yaml, yaml_file)